# **SIAM UQ24 MT5: PyDMD - Dynamic Mode Decomposition with Python**

**Abstract:** PyDMD is a Python package designed for Dynamic Mode Decomposition (DMD), a data-driven method used for analyzing and extracting spatiotemporal coherent structures from time-varying datasets. The package implements many extensions to handle noisy data, impose structure on the learned operators, and provide uncertainty estimates. PyDMD has a comprehensive and user-friendly interface, making it a valuable tool for researchers, engineers, and data scientists working in various fields. After this mini tutorial, participants will be able to use all the main functionalities of the package, exploit Bagging-Optimized Dynamic Mode Decomposition (BOP-DMD) for uncertainty quantification, and develop new DMD classes.

## Part 0: Installing PyDMD


In the last part of this minitutorial, we will create from scratch a new version of the original DMD, extending the standard `DMD` version. The goal is showing the steps to implement a new variant of DMD inside **PyDMD**, going deeper in the package structure.

Such an extension will be totally useless from a scientific perspective. But we did not want to create complex algorithms, we just wanted to show how to extend an already existing **PyDMD** class.

First of all, we install the right version of **PyDMD**:

```
pip install git+https://github.com/PyDMD/PyDMD
```
In this notebook, we already have the specific commit installation cell!

In [ ]:
#!pip install git+https://github.com/PyDMD/PyDMD.git@cbd7c923b2cc58ba1ab116b16295101adc256bbb

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from pydmd import DMDBase, DMD
from pydmd.utils import compute_tlsq
from pydmd.snapshots import Snapshots

import matplotlib.colors as colors

## Part 1: My First DMD variant

We are now able to create a new class **inheriting** from `DMDBase`: in this way, we just need to implement the new `fit()` method and all the other components are taken from the basic algorithm.
Of course, we can also overload one or more attributes/methods, it depends on the new algorithm we want to implement.

In this case, we also overload the constructor (i.e. the `__init__` method) to present a non-trivial case. 

The new functionality we are going to implement is a simple DMD that transforms (in some way) the input snapshots using a custom function, provided by the user before the construction of the operator.

Since the (quite) high number of parameters in the constructor, we suggest explicitly passing all of them instead of using `args` and `kwargs`. It's redundant, but also much more readable!
We can call the `DMDBase` constructor thanks to `super` keyword (more info on `super` [here](https://docs.python.org/3.9/library/functions.html?highlight=super#super)).

In [ ]:
class MyFancyNameDMD(DMDBase):
    """
    The MyFancyNameDMD class.
    A dummy DMD variant which, useless outside this tutorial.
    Make sure to write an exhaustive docstring if you want to
    distribute your code, or use it again in some years.
    """

    def __init__(self, func, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__func = func

We just specify that the last line of constructor saves the passed function in the object, giving us the possibility to recall it later.

We are now ready to implement the `fit` method. Define the custom mapping as $f: \mathbb R^n \to \mathbb R^n$ acting on the snapshots $\{\mathbf x_i \in \mathbb R^n\}_{i=1}^m$. Then we just need to pre-process the snapshots in `fit()` and build the DMD operator.

By recalling the already implemented methods, we just need few lines of code to complete the new version. Of course, in case of doubts, the [documentation](https://mathlab.github.io/PyDMD/index.html) is the best starting point to better understand classes and methods.

In [ ]:
class MyFancyNameDMD(DMDBase):
    """
    The MyFancyNameDMD class.
    A dummy DMD variant which, useless outside this tutorial.
    Make sure to write an exhaustive docstring if you want to
    distribute your code, or use it again in some years.
    """

    def __init__(self, func, **kwargs):
        super().__init__(**kwargs)
        self.__func = func

    def fit(self, X):
        """
        Compute the DMD to the input data.

        :param X: the input snapshots.
        :type X: numpy.ndarray or iterable
        """

        # adjust the shape of the snapshots
        input_snapshots = Snapshots(X).snapshots
        
        #####################################################
        # apply the mapping
        #####################################################
        mapped_snapshots = np.apply_along_axis(self.__func, 0, input_snapshots)
        
        # store the snapshots
        self._snapshots_holder = Snapshots(mapped_snapshots)
        
        # build the matrices X and Y
        n_samples = self.snapshots.shape[-1]
        X = self.snapshots[:, :-1]
        Y = self.snapshots[:, 1:]
        X, Y = compute_tlsq(X, Y, self._tlsq_rank)

        # compute the DMD operator
        self._svd_modes, _, _ = self.operator.compute_operator(X, Y)

        # Default timesteps
        self._set_initial_time_dictionary(
            {"t0": 0, "tend": n_samples - 1, "dt": 1}
        )

        # compute DMD amplitudes
        self._b = self._compute_amplitudes()

        return self

And, we're done!

Our new class is implemented, inheriting all the methods to the base class.
We have nothing to do but try it on a simple example. We build a simple dataset and we apply our new algorithm to it. We create the input data by summing two different functions and a costant value ($10^6$):
$$
\begin{align*}
    f_1(x,t) &= \text{sech}(x+3)\exp(i2.3t),\\
    f_2(x,t) &= 2\text{sech}(x)\tanh(x)\exp(i2.8t),
\end{align*}
$$
and finally $f(x,t) = f_1(x,t) + f_2(x,t) + 10^6$.

In [ ]:
def f1(x, t):
    return 1.0 / np.cosh(x + 3) * np.exp(2.3j * t)


def f2(x, t):
    return (2.0 / np.cosh(x) * np.tanh(x) * np.exp(2.8j * t))


x = np.linspace(-5, 5, 65)
t = np.linspace(0, 4 * np.pi, 129)

xgrid, tgrid = np.meshgrid(x, t)

X1 = f1(xgrid, tgrid)
X2 = f2(xgrid, tgrid)
X = X1 + X2 + 1e6

The dataset is ready, the only thing we need to do is instantiate our new class and call the `fit` method. We define the function `subtract` to be used as custom mapping: it just subtracts a fixed value to the snapshots matrix.

For a comparison, we also instantiate a standard DMD and call its `fit` method.

In [ ]:
def subtract(snapshot):
    return snapshot - 1e6


my_dmd = MyFancyNameDMD(func=subtract, svd_rank=2)
dmd = DMD(svd_rank=2)

[dmd_.fit(X.T) for dmd_ in [my_dmd, dmd]]

We have performed both the algorithms, we can now look at the reconstructed system using the `reconstructed_data` attribute. We note here that, as the other methods and attributes, `reconstructed_data` is inherited from `DMDBase`, giving us the possibility to call it without having implemented explicitly.
We prepare an helper function to plot the original data and the predicted results (for any passed DMD)!

In [ ]:
def compare_dmds(dmds, ground_truth):

    fig = plt.figure(layout=None, figsize=(18, 6*len(dmds)))
    gs = fig.add_gridspec(nrows=len(dmds)+2, ncols=2, left=0.05, right=0.95,
                      hspace=0.4, wspace=0.05)
    ax0 = fig.add_subplot(gs[:2, :])
    cb = ax0.pcolormesh(ground_truth.real.T)
    plt.colorbar(cb)

    for i, dmd_ in enumerate(dmds):
        ax1 = fig.add_subplot(gs[i+2, 0])
        ax2 = fig.add_subplot(gs[i+2, 1])
        cb = ax1.pcolormesh(dmd_.reconstructed_data.real)
        plt.title(f"{dmd_.__class__.__name__} Reconstruction")
        plt.xlabel("t")
        plt.ylabel("x")
        plt.colorbar(cb)
        
        Z = np.abs(dmd_.reconstructed_data.real - ground_truth.T)
        pcm = ax2.pcolormesh(
            Z,
            norm=colors.LogNorm(vmin=max(1.0e-16, Z.min()), vmax=max(1.0e-15, Z.max())),
        )
        ax2.set_title(f"{dmd_.__class__.__name__} Error")
        ax1.set_title(f"{dmd_.__class__.__name__} Reconstruction")

        plt.xlabel("t")
        plt.ylabel("x")
        plt.colorbar(pcm, extend="max")
    
    plt.show()

In [ ]:
compare_dmds([dmd, my_dmd], X)

Reconstruction is similar to the original data with the new variant, but the scales are completely different!
Contrarly, the standard DMD is able to reconstruct the original data with the same scale, but the original oscillations are not preserved.

What's going on?
We need to investigate a bit more!

#### Exercise 3a

Plot the (1D) modes for the new class, and compare them to the standard DMD.

#### Exercise 3b

Plot the dynamics for the new class, and compare them to the standard ones.

#### Exercise 3c

Plot the eigenvalues for the new class, and compare them to the standard ones.

## Part 2: Another, more complete, class

Ok, we have created a new class that looks better than the original one to catch the temporal trend with such the current dataset.
But we are not satisfied yet: we want to create a new class that is able to catch the temporal trend and preserve the correct scale of the original data.

#### Exercise 3d
Create a new DMD variants that takes two input (user-defined) functions:
- `transformation_func`: a function that transform the input snapshots;
- `inverse_transformation_func`: a function that transform the reconstructed data back to the original space.

Then, implement the `fit` method accordingly.

In [ ]:
class MyFancyName2DMD(MyFancyNameDMD):

    def __init__(self, inverse_transformation_func, transformation_func, *args, **kwargs):
        super().__init__(transformation_func, *args, **kwargs)
        self.__inverse_transformation_func = inverse_transformation_func

    # COMPLETE HERE

Great! Now it's time to test our new class. We will use the same dataset as before, and we will compare the results with the previous classes.
For the `MyFancyName2DMD` class, we will use the `subtract` function as `transformation_func` and `add` as `inverse_transformation_func`.

In [ ]:
def add(snapshots):
    return snapshots + 1e6
    
my2nd_dmd = MyFancyName2DMD(inverse_transformation_func=add, transformation_func=subtract, svd_rank=2)
my2nd_dmd.fit(X.T)

In [ ]:
compare_dmds([my2nd_dmd, my_dmd, dmd], X)

Amazing! Now the reconstruction is completely better that standard `DMD`. The scales are preserved and the oscillations are reconstructed in the right way.

#### Exercise 3e
What happen when you swap the functions we are passing?

Umh, we have a problem here. The reconstruction is not working as expected.
What is happening? We are now adding to the input snapshots the quantity $10^6$, applying DMD to these snapshots, and then subtracting $10^6$ to the reconstructed data. Instead of moving the original data closer to zero (like in previous case), we are making the data bigger, resulting in a bad conditioned problem.

With DMD (but also almost all the numerical algorithms), conditioning is a crucial aspect. We need to be careful when we are working with big numbers, and we need to be sure that the data we are working with is well conditioned. Let's try to see what is happening just making the date centered to zero.

In [ ]:
Xscaled = X - X.mean()

[dmd_.fit(Xscaled.T) for dmd_ in [my2nd_dmd, my_dmd, dmd]]
compare_dmds([my2nd_dmd, my_dmd, dmd], Xscaled)

Oh great! We can see a complete different scenario now. `DMD` is able to catch the correct behavior of the original data, and the new class we implemented is instead showing a bad reconstruction. The passed functions for pre- and post-processing the data are not pertinent to the current dataset: we can adapt such functions depending on the problem at hand, but can we obtain something more general?

#### Exercise 3f
Create a new version for making "zero_mean" preprocessing on the input data.

In [ ]:
compare_dmds([my2nd_dmd, my_dmd, center_dmd], X)

So remember, also center (and maybe scale) your data to reduce the numeric errors and instability! Even if the data are simple.

### Conclusion
We have shown a minimal extension of the basic `DMD` approach, hopefully demonstrating how easy can be to use **PyDMD** for further developments. Thanks to the object-oriented paradigm, it is possible to write (and test) a few lines of code, recycling all the components already implemented in the package! Be careful, always choose the *best* starting point.

To see an example where **PyDMD** is incorporated in some external frameworks, the page at [https://github.com/PyDMD/glappo/blob/master/notebooks/dldmd.ipynb]() introduces an application where **PyDMD** and **PyTorch** have been coupled to perform a nonlinear compression of the input data, increasing final accuracy!